In [31]:
import nltk
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

In [32]:
# load data
data = pd.read_csv('reddit_data.csv')
data.head()

,title,body,class
0,[AITA] I wrote an explanation in TIL and came ...,[Here is the post in question](http://www.redd...,True
1,[AITA] Threw my parent's donuts away,"My parents are diabetic, morbidly obese, and a...",True
2,I told a goth girl she looked like a clown.,I was four.,False
3,AITA Had a disagreement about Les Miserables w...,I love the musical *Les Miserables*. A coworke...,True
4,"[AITA] I 'hacked' our house router, changed th...",Backstory: I'm a semi-professional competitive...,True


In [33]:
sid = SentimentIntensityAnalyzer()
score = sid.polarity_scores('this is sentence is the shit')
score

{'neg': 0.404, 'neu': 0.449, 'pos': 0.146, 'compound': -0.5106}

In [34]:
body_sentiment_scores = [sid.polarity_scores(body) for body in data['body']]
title_sentiment_scores = [sid.polarity_scores(title) for title in data['title']]
#body_sentiment_scores

In [35]:
data['body_pos'] = [score['pos'] for score in body_sentiment_scores]
data['body_neg'] = [score['neg'] for score in body_sentiment_scores]
data['body_neu'] = [score['neu'] for score in body_sentiment_scores]
data['body_comp'] = [score['compound'] for score in body_sentiment_scores]

data['title_pos'] = [score['pos'] for score in title_sentiment_scores]
data['title_neg'] = [score['neg'] for score in title_sentiment_scores]
data['title_neu'] = [score['neu'] for score in title_sentiment_scores]
data['title_comp'] = [score['compound'] for score in title_sentiment_scores]

data.to_csv('sentiment-data.csv', index=False)
data.head()

,title,body,class,body_pos,body_neg,body_neu,body_comp,title_pos,title_neg,title_neu,title_comp
0,[AITA] I wrote an explanation in TIL and came ...,[Here is the post in question](http://www.redd...,True,0.169,0.068,0.763,0.7692,0.000,0.000,1.000,0.0000
1,[AITA] Threw my parent's donuts away,"My parents are diabetic, morbidly obese, and a...",True,0.088,0.050,0.862,0.8591,0.000,0.000,1.000,0.0000
2,I told a goth girl she looked like a clown.,I was four.,False,0.000,0.000,1.000,0.0000,0.294,0.000,0.706,0.3612
3,AITA Had a disagreement about Les Miserables w...,I love the musical *Les Miserables*. A coworke...,True,0.132,0.054,0.814,0.9075,0.000,0.263,0.737,-0.3612
4,"[AITA] I 'hacked' our house router, changed th...",Backstory: I'm a semi-professional competitive...,True,0.077,0.048,0.875,0.9174,0.000,0.000,1.000,0.0000


In [5]:
data = pd.read_csv('sentiment-data.csv')
data.head()

,title,body,class,body_pos,body_neg,body_neu,body_comp,title_pos,title_neg,title_neu,title_comp
0,[AITA] I wrote an explanation in TIL and came ...,[Here is the post in question](http://www.redd...,True,0.169,0.068,0.763,0.7692,0.000,0.000,1.000,0.0000
1,[AITA] Threw my parent's donuts away,"My parents are diabetic, morbidly obese, and a...",True,0.088,0.050,0.862,0.8591,0.000,0.000,1.000,0.0000
2,I told a goth girl she looked like a clown.,I was four.,False,0.000,0.000,1.000,0.0000,0.294,0.000,0.706,0.3612
3,AITA Had a disagreement about Les Miserables w...,I love the musical *Les Miserables*. A coworke...,True,0.132,0.054,0.814,0.9075,0.000,0.263,0.737,-0.3612
4,"[AITA] I 'hacked' our house router, changed th...",Backstory: I'm a semi-professional competitive...,True,0.077,0.048,0.875,0.9174,0.000,0.000,1.000,0.0000


In [40]:
x = data.drop(['title', 'body', 'class'], axis=1)
y = data['class']
x.head()

,body_pos,body_neg,body_neu,body_comp,title_pos,title_neg,title_neu,title_comp
0,0.169,0.068,0.763,0.7692,0.000,0.000,1.000,0.0000
1,0.088,0.050,0.862,0.8591,0.000,0.000,1.000,0.0000
2,0.000,0.000,1.000,0.0000,0.294,0.000,0.706,0.3612
3,0.132,0.054,0.814,0.9075,0.000,0.263,0.737,-0.3612
4,0.077,0.048,0.875,0.9174,0.000,0.000,1.000,0.0000


In [45]:
# run SMOTE
print(len(x))
sm = SMOTE('minority')
x, y = sm.fit_resample(x, y)
print(len(x))
print(list(y).count(True))

3490
3490
1745


In [46]:
knn = KNeighborsClassifier(n_neighbors=5)
cv = cross_val_score(knn, x, y, cv=10)
print(cv)
y_pred = cross_val_predict(knn, x, y, cv=10)
report = classification_report(y, y_pred)
print(report)

[0.61891117 0.62750716 0.61318052 0.63896848 0.66189112 0.7277937
 0.70487106 0.71060172 0.73352436 0.72206304]
              precision    recall  f1-score   support

       False       0.73      0.56      0.63      1745
        True       0.64      0.79      0.71      1745

    accuracy                           0.68      3490
   macro avg       0.69      0.68      0.67      3490
weighted avg       0.69      0.68      0.67      3490



In [47]:
param_grid = {'n_neighbors': [i for i in range(1,31)]}
gridcv = GridSearchCV(knn, param_grid)
gridcv.fit(x, y)
print(gridcv.best_params_)
gridcv.best_score_
y_pred = cross_val_predict(gridcv, x, y, cv=10)
report = classification_report(y, y_pred)
print(report)

{'n_neighbors': 1}
              precision    recall  f1-score   support

       False       0.84      0.67      0.74      1745
        True       0.72      0.87      0.79      1745

    accuracy                           0.77      3490
   macro avg       0.78      0.77      0.77      3490
weighted avg       0.78      0.77      0.77      3490



In [56]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
pca = PCA(n_components = 0.95)
pca.fit(x_train)
x_train_pca = pca.transform(x_train)
print(x_train_pca.shape)

# apply to test set
x_test_pca = pca.transform(x_test)
print(x_test.shape)

(2792, 3)
(698, 8)


In [57]:
param_grid = {'n_neighbors': [i for i in range(1,31)]}
gridcv = GridSearchCV(knn, param_grid)
gridcv.fit(x_train, y_train)
print(gridcv.best_params_)
gridcv.best_score_

{'n_neighbors': 1}


0.7260065016254064

In [58]:
tree = DecisionTreeClassifier(criterion='entropy')

param_grid = {
    'max_depth': range(1,16),
    'min_samples_leaf': range(5,40,5),
}
gridcv = GridSearchCV(tree, param_grid)
gridcv.fit(x_train, y_train)
print(gridcv.best_params_)
gridcv.best_score_

{'max_depth': 15, 'min_samples_leaf': 30}


0.5816633645590885

In [62]:
rand_for = RandomForestClassifier()
param_grid = {
    'max_depth': range(4,10),
    'min_samples_leaf': range(1,5),
    'max_features': ('sqrt', 'log2')
}
gridcv = GridSearchCV(rand_for, param_grid)
gridcv.fit(x_train, y_train)
print(gridcv.best_params_)
gridcv.best_score_

{'max_depth': 9, 'max_features': 'log2', 'min_samples_leaf': 8}


0.6096094536454626